In [1]:
#Import checks
from pip._internal import main as pipmain

try:
    from nltk.tokenize import word_tokenize, sent_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
except:
    pipmain(["install", "nltk"])
    from nltk.tokenize import word_tokenize, sent_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
try:
    import re
except:
    pipmain(["install", "re"])
    import re
try:
    import string
except:
    pipmain(["install", "string"])
    import string
try:
    import unicodedata
except:
    pipmain(["install", "unicodedata"])
    import unicodedata
try:
    import pandas
except:
    pipmain(["install", "pandas"])
    import pandas
try:
    import os
except:
    pipmain(["install", "os"])
    import os
try:
    import collections
except:
    pipmain(["install", "collections"])
    import collections
try:
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
except:
    pipmain(["install", "sklearn"])
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
try:
    import tweepy
except:
    pipmain(["install", "tweepy"])
    import tweepy
try:
    import csv
except:
    pipmain(["install", "csv"])
    import csv
try:
    import time
except:
    pipmain(["install", "time"])
    import time
try:
    import pickle
except:
    pipmain(["install", "pickle"])
    import pickle


In [2]:
import nltk
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer
import unicodedata
default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english') # or any other list of your choice
def TweetbScrub(text, ):
    if text[0] == "b":
        text = text[1:]
    return text

def clean_text(text, ):

    def tokenize_text(text):
        return [w for s in sent_tokenize(text) for w in word_tokenize(s)]

    def remove_special_characters(text, characters = string.punctuation.replace('-', '')):
        tokens = tokenize_text(text)
        pattern = re.compile('[{}]'.format(re.escape(characters)))
        return ' '.join(filter(None, [pattern.sub('', t) for t in tokens]))

    def stem_text(text, stemmer=default_stemmer):
        tokens = tokenize_text(text)
        return ' '.join([stemmer.stem(t) for t in tokens])

    def remove_stopwords(text, stop_words=default_stopwords):
        tokens = [w for w in tokenize_text(text) if w not in stop_words]
        return ' '.join(tokens)

    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'[^^a-zA-Z ]', '', text)
    text = text.strip(' ') # strip whitespaces
    text = text.lower() # lowercase
    text = stem_text(text) # stemming
    try:
        text = remove_special_characters(text) # remove punctuation and symbols
    except:
        print("Error")
    text = remove_stopwords(text) # remove stopwords
    #text.strip(' ') # strip whitespaces again?

    return text

PartyColours = {"Labour": "#dc241f", 
              "Conservative": "#0087dc", 
              "SNP": "#fff95d", 
              "Green": "#6ab023", 
              "UKIP": "#70147a", 
              "DUP":"#d46a4c", 
              "Non-Aligned": "#ffffff", 
              "LibDem": "#fdbb30"}

#Twitter API credentials
consumer_key = "TC98w89JxQK2v4vPEqLLxJLx0"
consumer_secret = "le4t2JCgoT3CBZwToaKdOJx5LFYDDkGL5e3Pjl2ZtfTqYV46Fs"
access_key = "4459846396-tU9aYf4E5r9eHhJnniU7OsyrLNJhzEd4cpVeFFe"
access_secret = "UaY6kpdXbdV7cAsAxrKLzFTkKSLtW8dcNTe1CYniUl6xM"



def get_all_tweets(screen_name):
    
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))

    #transform the tweepy tweets into a 2D array that will populate the csv



    outtweets = []
    for tweet in alltweets:
        media_count = 0
        url_count =0
        if "media" in tweet.entities:
            media_count =1
        if "urls" in tweet.entities:
            url_count =len(tweet.entities["urls"])
        outtweets.append([tweet.id_str, tweet.created_at, tweet.text.encode("utf-8"),tweet.retweet_count,tweet.favorite_count,media_count,url_count])

    #write the csv
    with open(screen_name + '.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text","retweet_count","favorite_count","media_count","url_count"])
        writer.writerows(outtweets)

    pass

def GeneratePie(collection, name):
    import plotly 
    plotly.tools.set_credentials_file(username='andriy93', api_key='j4UgdivFmCTlwkO1bYWS')

    import plotly.plotly as py
    import plotly.graph_objs as go


    labels = list(collection.keys())
    values = list(collection.values())
    #colors = [PartyColours[i] for i in labels]


    trace = go.Pie(labels=labels, values=values,
                   hoverinfo='label+percent', textinfo='value', 
                   textfont=dict(size=10),
                   marker=dict(
                               line=dict(color='#000000', width=2)))

    data = [trace]
    layout = go.Layout(title=name)
    fig = go.Figure(data=data,layout=layout)

    # Save the figure as a png image:
    py.image.save_as(fig, name + '.png')


def AnalyseSpeech(Filename):
    os.chdir("E:\Honours\Master\Speeches")
    Filename = Filename.replace(".txt", "")
    Speech = open(Filename + ".txt").read()
    Sentence = vectorizer.transform([Speech])
    classifier.predict(Sentence)
    Speech = Speech.replace("\n", "")
    Speech = Speech.split(".")
    for sentence in Speech:
        if sentence == " ":
            Speech.remove(sentence)
    Speech = list(filter(None, Speech))

    Speech = pandas.DataFrame(Speech)
    Speech["Classification"] = ""
    Speech.columns = ['Text', "Classification"]
    Speech['CleanText'] = Speech['Text'].apply(lambda row: clean_text(row))
    Sentence = vectorizer.transform(Speech["CleanText"])
    Classification = classifier.predict(Sentence)
    Speech["Classification"] = Classification
    Speech = Speech.apply(Sentimentclassifer,axis=1)
    Speech.drop('CleanText', axis=1, inplace=True)
    Speechcollection = collections.Counter(Speech["Classification"])
    #Change directory to the classifications directory for saving
    os.chdir("E:\Honours\Master\Speeches\Classifications\British")
    #Save Classifications to classificaiton directory
    try:
        os.mkdir(Filename)
    except:
        pass
    os.chdir(Filename)
    Speech.to_csv(Filename + "Classification.csv", encoding='utf-8')

    GeneratePie(Speechcollection, Filename)

def AnalyseUser(Username):
    os.chdir("E:\Honours\Master\Twitter-User-Data\Classifications")
    try:
        os.mkdir(Username)
    except:
        pass
    os.chdir(Username)
    get_all_tweets(Username)
    Username = Username.replace(".csv", "")
    UserData = pandas.DataFrame.from_csv(Username + ".csv")
    #Vectorize the text
    UserData['CleanText'] = UserData['text'].apply(lambda row: clean_text(row))
    Sentences = vectorizer.transform(UserData['CleanText'])
    #Perform prediction of text inputted
    Classification = classifier.predict(Sentences)
    #Output a basic overview of the predictions
    #Append original data with model classification
    UserData["Classification"] = Classification
    UserData = UserData.apply(Sentimentclassifer,axis=1)
    UserData.drop('CleanText', axis=1, inplace=True)
    Usercollection = collections.Counter(UserData["Classification"])
    #Change directory to the classifications directory for saving
    #Save Classifications to classificaiton directory
    UserData.to_csv(Username + "-Classification.csv", encoding='utf-8')
    os.remove(Username + ".csv")
    
    
    GeneratePie(Usercollection, Username)

def Sentimentclassifer(sentence, ):
    sentiment = sid.polarity_scores(sentence['CleanText'])
    sentence['Sentiment'] = sentiment['compound']
    if (sentence['Sentiment'] < -0) & (sentence['Classification'] != "Non-Aligned"):
        sentence['Classification'] = "Anti-" + sentence['Classification']
    return sentence

In [47]:
try:
    os.chdir("E:\Honours\Master\Model")
    test = pickle.load(open("Model", 'rb'))
    BritishDF = pandas.DataFrame.from_csv("BritishCleaned.csv")
    vectorizer = CountVectorizer(min_df=0, lowercase=False)
    vectorizer.fit(BritishDF['text'])
except:
    os.chdir("E:\Honours\Master\Twitter-Learning-Data\Collections")

    BritishDF = pandas.DataFrame.from_csv("BritishCollection.csv")

    BritishDF['text'] = BritishDF['text'].apply(lambda row: clean_text(row))

    vectorizer = CountVectorizer(min_df=0, lowercase=False)

    vectorizer.fit(BritishDF['text'])

    sentences = BritishDF['text'].values
    y = BritishDF['Alignment'].values

    sentences_train, sentences_test, y_train, y_test = train_test_split(
        sentences, y, test_size=0.05, random_state=1000)

    vectorizer = CountVectorizer()
    vectorizer.fit(sentences_train)

    X_train = vectorizer.transform(sentences_train)
    X_test  = vectorizer.transform(sentences_test)

    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)

    print("Accuracy:", score)

    os.chdir("E:\Honours\Master\Model")
    pickle.dump(classifier, open("Model", 'wb'))
    BritishDF.to_csv("BritishCleaned.csv", encoding='utf-8')

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  # Remove the CWD from sys.path while we load stuff.


Accuracy: 0.8103104405133541


NameError: name 'pickle' is not defined

In [ ]:
os.chdir("E:\Honours\Master\Twitter-User-Data\Raw")

for filename in os.listdir():
        if ".csv" in filename: 
            AnalyseUser(filename)
            


In [118]:
os.chdir("E:\Honours\Master\Speeches")
for filename in os.listdir():
        if ".txt" in filename: 
            AnalyseSpeech(filename)
            
        


In [78]:
uesrname = input("Username:")
AnalyseUser(uesrname)

Username:GuyFieriisJesus
getting tweets before 598091020547583999
...32 tweets downloaded so far


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:192: FutureWarning:

from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls



In [109]:
AnalyseUser("GuyFieriisJesus")

getting tweets before 598091020547583999
...32 tweets downloaded so far


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:193: FutureWarning:

from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls



In [117]:
AnalyseSpeech("Hitler-Danzig")

In [75]:
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
def Tweetclassifer(tweet, ):
    sentiment = sid.polarity_scores(tweet['CleanText'])
    tweet['Sentiment'] = sentiment['compound']
    return tweet
UserDatad = UserData.apply(Tweetclassifer,axis=1)


In [76]:
UserDatad
os.chdir("E:\Honours\Master\Model")
UserDatad.to_csv("test.csv", encoding='utf-8')